In [ ]:
import os
import math
import gc

import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.linear_model import SGDRegressor, LinearRegression, Ridge
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')

from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding, LSTM
from keras.utils import to_categorical

In [2]:
data_path = '/Users/chiang/数据/天池-心跳信号分类预测/'
data_train = pd.read_csv(data_path + 'train.csv')
data_test_A = pd.read_csv(data_path + 'testA.csv')

In [3]:
data_train

,id,heartbeat_signals,label
0,0,"0.9912297987616655,0.9435330436439665,0.764677...",0.0
1,1,"0.9714822034884503,0.9289687459588268,0.572932...",0.0
2,2,"1.0,0.9591487564065292,0.7013782792997189,0.23...",2.0
3,3,"0.9757952826275774,0.9340884687738161,0.659636...",0.0
4,4,"0.0,0.055816398940721094,0.26129357194994196,0...",2.0
...,...,...,...
99995,99995,"1.0,0.677705342021188,0.22239242747868546,0.25...",0.0
99996,99996,"0.9268571578157265,0.9063471198026871,0.636993...",2.0
99997,99997,"0.9258351628306013,0.5873839035878395,0.633226...",3.0
99998,99998,"1.0,0.9947621698382489,0.8297017704865509,0.45...",2.0


In [4]:
train_data_total = []
for items in data_train.values:
    train_data_total.append([float(i) for i in items[1].split(',')])

In [5]:
print(train_data_total[0])

[0.9912297987616655, 0.9435330436439665, 0.7646772997256593, 0.6185708990212999, 0.3796321642826237, 0.19082233510621885, 0.040237131594430715, 0.02599520771717858, 0.03170886048677242, 0.06552357497104398, 0.12553088566683082, 0.14674736762087337, 0.16765635354203254, 0.19337353075154495, 0.22613482558418235, 0.2211427948707646, 0.23606736350657742, 0.2211427948707646, 0.2211427948707646, 0.21110661221417562, 0.20858662883955462, 0.19337353075154495, 0.19592021355822875, 0.1984624088145674, 0.18570638844539308, 0.19592020417425474, 0.18314160533045887, 0.19337353075154495, 0.19082233510621885, 0.20858662883955462, 0.2211427948707646, 0.2508391000672623, 0.2606035735248363, 0.27753397418529446, 0.2942679945470305, 0.3037438606924122, 0.3364276621747203, 0.3479233126336631, 0.38410562561692113, 0.3863371817756788, 0.4084648300418338, 0.4106590521686313, 0.42592580507675887, 0.42592580507675887, 0.4291763526701312, 0.4324195928902589, 0.42809365036122277, 0.42809365036122277, 0.412849942

In [7]:
print("共有 {} 个样本".format(len(train_data_total)))
print("每个样本维度为：{}".format(len(train_data_total[0])))

data_dim = len(train_data_total[0])
for idx in train_data_total:
    if len(idx) != data_dim:
        print("维度不统一")
    

共有 100000 个样本
每个样本维度为：205


In [8]:
print(type(train_data_total))

<class 'list'>


In [10]:
train_data_total = np.array(train_data_total)
print(train_data_total.shape)

(100000, 205)


In [11]:
test_x = []
for items in data_test_A.values:
    test_x.append([float(i) for i in items[1].split(',')])

In [12]:
test_x = np.array(test_x)
print(test_x.shape)

(20000, 205)


In [13]:
train_label = data_train.label
train_label

0        0.0
1        0.0
2        2.0
3        0.0
4        2.0
        ... 
99995    0.0
99996    2.0
99997    3.0
99998    2.0
99999    0.0
Name: label, Length: 100000, dtype: float64

In [14]:
temp = []
for idx in train_label:
    temp.append(int(idx))

In [15]:
train_label = temp
print(len(train_label))

100000


In [16]:
train_label = to_categorical(train_label)
train_label.shape

(100000, 4)

In [17]:
model = Sequential()
model.add(Dense(128, input_shape=(205,), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               26368     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 260       
Total params: 34,884
Trainable params: 34,884
Non-trainable params: 0
_________________________________________________________________


In [18]:
train_x = train_data[1000:]
train_y = train_label[1000:]

val_x = train_data[:1000]
val_y = train_label[:1000]

print("全量数据维度为: {}".format(train_data.shape))
print("全量标签维度为: {}".format(train_label.shape))

print("训练集数据维度为: {}".format(train_x.shape))
print("训练集标签维度为: {}".format(train_y.shape))

print("验证集数据维度为: {}".format(val_x.shape))
print("验证集标签维度为: {}".format(val_y.shape))

print("测试集数据维度为: {}".format(test_x.shape))


全量数据维度为: (100000, 205)
全量标签维度为: (100000, 4)
训练集数据维度为: (99000, 205)
训练集标签维度为: (99000, 4)
验证集数据维度为: (1000, 205)
验证集标签维度为: (1000, 4)
测试集数据维度为: (20000, 205)


In [20]:
print(train_x[0,1])
print(type(train_x[0, 1]))

0.8164311229285389
<class 'numpy.float64'>


In [23]:
history = model.fit(train_x, train_y, epochs=10, batch_size=512, validation_data=(val_x, val_y))

Epoch 1/10
194/194 [==============================] - 1s 5ms/step - loss: 0.2766 - accuracy: 0.9102 - val_loss: 0.2387 - val_accuracy: 0.9240
Epoch 2/10
194/194 [==============================] - 1s 4ms/step - loss: 0.2185 - accuracy: 0.9287 - val_loss: 0.2193 - val_accuracy: 0.9320
Epoch 3/10
194/194 [==============================] - 1s 4ms/step - loss: 0.1862 - accuracy: 0.9400 - val_loss: 0.1866 - val_accuracy: 0.9450
Epoch 4/10
194/194 [==============================] - 1s 5ms/step - loss: 0.1627 - accuracy: 0.9483 - val_loss: 0.2274 - val_accuracy: 0.9320
Epoch 5/10
194/194 [==============================] - 1s 4ms/step - loss: 0.1472 - accuracy: 0.9537 - val_loss: 0.1727 - val_accuracy: 0.9530
Epoch 6/10
194/194 [==============================] - 1s 4ms/step - loss: 0.1341 - accuracy: 0.9585 - val_loss: 0.1581 - val_accuracy: 0.9590
Epoch 7/10
194/194 [==============================] - 1s 4ms/step - loss: 0.1244 - accuracy: 0.9619 - val_loss: 0.1591 - val_accuracy: 0.9570
Epoch 

In [24]:
result = model.predict(test_x)

In [25]:
print(result.shape)

(20000, 4)


In [26]:
for idx in range(100):
    print(result[idx])

[9.9963677e-01 3.5775616e-04 3.0700093e-07 5.0657304e-06]
[2.9952184e-03 1.3015603e-01 8.6677253e-01 7.6280994e-05]
[1.2548773e-04 2.6857887e-08 3.7276038e-06 9.9987078e-01]
[9.9654323e-01 3.3909779e-03 6.5454566e-05 3.9518542e-07]
[9.9926692e-01 2.7985231e-04 1.3308908e-04 3.2014953e-04]
[9.9676067e-01 2.9320642e-03 3.0678092e-04 5.2879926e-07]
[9.7304779e-01 4.5178416e-03 5.0572579e-04 2.1928633e-02]
[9.9722606e-01 1.0850997e-04 6.1453541e-04 2.0508636e-03]
[9.8210943e-01 1.7878182e-02 7.5212693e-06 4.9665496e-06]
[1.1365634e-05 8.6426921e-10 1.9801222e-08 9.9998856e-01]
[0.9410128  0.05331648 0.00445403 0.00121665]
[4.5881402e-01 9.6853338e-03 5.3145224e-01 4.8428468e-05]
[9.3813014e-01 2.1058887e-02 4.0297046e-02 5.1388692e-04]
[9.8843342e-01 1.0593704e-02 9.7196968e-04 9.5455039e-07]
[9.9653554e-01 3.3723935e-03 5.9425489e-05 3.2667271e-05]
[9.9276137e-01 7.1913507e-03 4.7390837e-05 4.5157105e-08]
[9.9951780e-01 4.7871226e-04 1.9115082e-07 3.1927116e-06]
[2.9973080e-04 5.0644617e-

In [27]:
result[0]

array([9.9963677e-01, 3.5775616e-04, 3.0700093e-07, 5.0657304e-06],
      dtype=float32)

In [30]:
final_result = []
for idx in range(len(result)):
    final_result.append(np.argmax(result[idx]))

In [31]:
final_result

[0,
 2,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 3,
 0,
 3,
 0,
 3,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 3,
 0,
 3,
 0,
 2,
 0,
 1,
 3,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 3,
 3,
 0,
 1,
 0,
 3,
 3,
 0,
 2,
 3,
 2,
 0,
 0,
 0,
 0,
 0,
 3,
 1,
 0,
 2,
 3,
 3,
 3,
 0,
 0,
 3,
 0,
 0,
 3,
 0,
 3,
 0,
 1,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 2,
 0,
 3,
 0,
 3,
 0,
 0,
 0,
 0,
 1,
 2,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 3,
 0,
 0,
 3,
 3,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 3,
 0,
 2,
 3,
 2,
 3,
 0,
 2,
 2,
 3,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 2,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 3,


In [34]:
temp = to_categorical(final_result).tolist()
temp

[[1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [1.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0],
 [1.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0],
 [1.0, 0.0,

In [35]:
result = []
result_temp0 = []
result_temp1 = []
result_temp2 = []
result_temp3 = []
for idx in range(len(temp)):
    result_temp0.append(temp[idx][0])
    result_temp1.append(temp[idx][1])
    result_temp2.append(temp[idx][2])
    result_temp3.append(temp[idx][3])
result.append(result_temp0)
result.append(result_temp1)
result.append(result_temp2)
result.append(result_temp3)
print(result)

[[1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0

In [39]:
result_data = pd.read_csv(data_path + 'sample_submit.csv')

result_data['label_0']=result_temp0
result_data['label_1']=result_temp1
result_data['label_2']=result_temp2
result_data['label_3']=result_temp3
result_data.to_csv('./20210317_submit.csv',index=False)